In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import cvxopt as opt
from cvxopt import blas, solvers
from sklearn.linear_model import LinearRegression

In [2]:
#Import CSVs
ABT = pd.read_csv('/Users/BLucey/Desktop/Coding/Codeacademy/Python/FA/Capstone/CSV/ABT.csv')
DIV = pd.read_csv('/Users/BLucey/Desktop/Coding/Codeacademy/Python/FA/Capstone/CSV/DIV.csv')
JPM = pd.read_csv('/Users/BLucey/Desktop/Coding/Codeacademy/Python/FA/Capstone/CSV/JPM.csv')
O = pd.read_csv('/Users/BLucey/Desktop/Coding/Codeacademy/Python/FA/Capstone/CSV/O.csv')
TSLA = pd.read_csv('/Users/BLucey/Desktop/Coding/Codeacademy/Python/FA/Capstone/CSV/TSLA.csv')
TSM = pd.read_csv('/Users/BLucey/Desktop/Coding/Codeacademy/Python/FA/Capstone/CSV/TSM.csv')

print(ABT.head())

      Date       Open       High        Low      Close  Adj Close    Volume
0  6/24/19  84.779999  85.330002  84.639999  84.709999  83.350471   4531600
1  6/25/19  84.940002  85.449997  84.239998  84.239998  82.888008   4915300
2  6/26/19  84.050003  84.370003  82.629997  82.959999  81.628563   5731400
3  6/27/19  83.389999  83.980003  83.070000  83.690002  82.346848   4111600
4  6/28/19  83.820000  84.260002  83.120003  84.099998  82.750259  16436700


In [3]:
#Parse out unnecessary columns
ABT_1 = ABT[['Date','Adj Close']]
ABT_1.rename(columns = {'Adj Close' : 'ABT'}, inplace = True)
print(ABT_1.head())


DIV_1 = DIV[['Adj Close']]
DIV_1.rename(columns = {'Adj Close' : 'DIV'}, inplace = True)
print(DIV_1.head())


JPM_1 = JPM[['Adj Close']]
JPM_1.rename(columns = {'Adj Close' : 'JPM'}, inplace = True)
print(JPM_1.head())


O_1 = O[['Adj Close']]
O_1.rename(columns = {'Adj Close' : 'O'}, inplace = True)
print(O_1.head())

            
TSLA_1 = TSLA[['Adj Close']]
TSLA_1.rename(columns = {'Adj Close' : 'TSLA'}, inplace = True)
print(TSLA_1.head())


TSM_1 = TSM[['Adj Close']]
TSM_1.rename(columns = {'Adj Close' : 'TSM'}, inplace = True)
print(TSM_1.head())



      Date        ABT
0  6/24/19  83.350471
1  6/25/19  82.888008
2  6/26/19  81.628563
3  6/27/19  82.346848
4  6/28/19  82.750259
         DIV
0  20.573940
1  20.546385
2  20.426981
3  20.601498
4  20.803556
          JPM
0  105.262070
1  104.390221
2  105.087700
3  105.436440
4  108.303879
           O
0  68.945511
1  67.629623
2  65.795044
3  66.678711
4  66.463326
         TSLA
0  223.639999
1  219.759995
2  219.270004
3  222.839996
4  223.460007
         TSM
0  38.758575
1  37.730961
2  37.886074
3  38.380489
4  37.973324


/opt/miniconda3/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/opt/miniconda3/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/opt/miniconda3/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/opt/miniconda3/lib/python3.7/site-packages

In [4]:
#New comprehensive dataframe
ABT_1.insert(2, 'DIV', DIV_1)
ABT_1.insert(3, 'JPM', JPM_1)
ABT_1.insert(4, 'O', O_1)
ABT_1.insert(5, 'TSLA', TSLA_1)
ABT_1.insert(6, 'TSM', TSM_1)
print(ABT_1)

#Change Date to Index
df = ABT_1.set_index('Date')

print(df.info())


        Date        ABT        DIV         JPM          O         TSLA  \
0    6/24/19  83.350471  20.573940  105.262070  68.945511   223.639999   
1    6/25/19  82.888008  20.546385  104.390221  67.629623   219.759995   
2    6/26/19  81.628563  20.426981  105.087700  65.795044   219.270004   
3    6/27/19  82.346848  20.601498  105.436440  66.678711   222.839996   
4    6/28/19  82.750259  20.803556  108.303879  66.463326   223.460007   
..       ...        ...        ...         ...        ...          ...   
249  6/18/20  90.279999  15.660000   98.940002  63.389999  1003.960022   
250  6/19/20  90.680000  15.490000   97.809998  61.529999  1000.900024   
251  6/22/20  90.339996  15.450000   96.750000  61.439999   994.320007   
252  6/23/20  91.300003  15.390000   97.930000  61.270000  1001.780029   
253  6/24/20  87.190002  14.940000   94.669998  58.509998   963.289978   

           TSM  
0    38.758575  
1    37.730961  
2    37.886074  
3    38.380489  
4    37.973324  
..       

In [5]:
#Expected Returns
daily_ER = df.pct_change()
print(daily_ER)

AVG_daily_ER = daily_ER.mean()
print(AVG_daily_ER)

              ABT       DIV       JPM         O      TSLA       TSM
Date                                                               
6/24/19       NaN       NaN       NaN       NaN       NaN       NaN
6/25/19 -0.005548 -0.001339 -0.008283 -0.019086 -0.017349 -0.026513
6/26/19 -0.015195 -0.005811  0.006681 -0.027127 -0.002230  0.004111
6/27/19  0.008799  0.008543  0.003319  0.013431  0.016281  0.013050
6/28/19  0.004899  0.009808  0.027196 -0.003230  0.002782 -0.010609
...           ...       ...       ...       ...       ...       ...
6/18/20 -0.006821  0.000639 -0.005428  0.005233  0.012271 -0.000534
6/19/20  0.004431 -0.010856 -0.011421 -0.029342 -0.003048 -0.014960
6/22/20 -0.003749 -0.002582 -0.010837 -0.001463 -0.006574  0.012475
6/23/20  0.010627 -0.003883  0.012196 -0.002767  0.007503  0.013571
6/24/20 -0.045016 -0.029240 -0.033289 -0.045047 -0.038422 -0.002290

[254 rows x 6 columns]
ABT     0.000453
DIV    -0.000895
JPM     0.000080
O       0.000011
TSLA    0.006957
TSM    

In [6]:
#Calculate Covariance
covariance = daily_ER.cov()
print(covariance)

           ABT       DIV       JPM         O      TSLA       TSM
ABT   0.000553  0.000299  0.000448  0.000356  0.000341  0.000327
DIV   0.000299  0.000709  0.000693  0.000783  0.000630  0.000375
JPM   0.000448  0.000693  0.001002  0.000787  0.000587  0.000518
O     0.000356  0.000783  0.000787  0.001250  0.000831  0.000443
TSLA  0.000341  0.000630  0.000587  0.000831  0.002347  0.000552
TSM   0.000327  0.000375  0.000518  0.000443  0.000552  0.000596


In [7]:
#Define Return Portfolios
def return_portfolios(expected_returns, cov_matrix):
    port_returns = []
    port_volatility = []
    stock_weights = []

    selected = (expected_returns.axes)[0]

    num_assets = len(selected)
    num_portfolios = 5000

    for single_portfolio in range(num_portfolios):
        weights = np.random.random(num_assets)
        weights /= np.sum(weights)
        returns = np.dot(weights, expected_returns)
        volatility = np.sqrt(np.dot(weights.T,np.dot(cov_matrix, weights)))
        port_returns.append(returns)
        port_volatility.append(volatility)

        stock_weights.append(weights)

    portfolio = {'Returns': port_returns, 'Volatility': port_volatility}

    for counter, symbol in enumerate(selected):
        portfolio[symbol + 'Weight'] = [Weight [counter] for Weight in stock_weights]

        df = pd.Dataframe(portfolio)

        column_order = ['Returns', 'Volatility'] + [stock +' Weight' for stock in selected]

        df = df[column_order]

        return df


In [8]:
#Define Optimal Portfolio
def optimal_portfolio(returns):
    n = returns.shape[1]
    returns = np.transpose(returns.as_matrix())

    N = 100
    mus = [10 ** (5.0 * t/N - 1.0) for t in range (N)]

    #Convert to cvxopt matrices
    S = opt.matrix(np.cov(returns))
    pbar = opt.matrix(np.mean(returns, axis = 1))

    #Create constraint matrices
    G = -opt.matrix(np.eye(n)) #negative n x n identity matrix
    h = opt.matrix(0.0, (n, 1))
    A = opt.matrix(1.0, (1, n))
    b = opt.matrix(1.0)

    #Calculate efficient frontier weights using quadratic programming
    portfolios = [solvers.qp(mu * S, -pbar, G, h, A, b)['x'] for mu in mus]

    ##Calculate the 2nd degree polynomial of the frontier curve
    m1 = np.polyfit(returns, risks, 2)
    x1 = np.sqrt(m1[2] / m1[0])

    #Calculate the optimal portfolio
    wt = solvers.qp(opt.matrix(x1 * S), -pbar, G, h, A, b)['x']
    return np.asarray(wt), returns, risks


In [9]:
#Plot Efficient Frontier
df_1 = return_portfolios(daily_ER, covariance)

weights, returns, risks = optimal_portfolio(daily_ER[1:])

Data.plot.scatter(x = 'Volatility', y = 'Returns', fontsize = 12)
plt.plot(risks, returns, 'y-o')
plt.scatter(single_asset_std, daily_ER, marker = 'X', color = 'orange', s = 200)
for xc in single_asset_std:
    plt.axvline(x = xc, color = 'orange')


ValueError: shapes (6,6) and (254,) not aligned: 6 (dim 1) != 254 (dim 0)